<a href="https://colab.research.google.com/github/itzdeenzxx/DeepkSeek_R1_1_5B_Finetuning/blob/main/DeepkSeek_R1_1_5B_Finetuning_LoRA_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/12nB0-QC5DU9hL79Q9eBtXEXLMEvAYMpk?usp=sharing <br>
https://www.linkedin.com/pulse/fine-tune-deepseek-r1-15b-free-gcp-colab-t4-hands-on-konathala-phd--4bluf/

***
Approach 1: End-to-End Code for Fine-Tuning DeepSeek R1 1.5B for Domain-Specific Text Generation
***
By Thirumalesh Konathala

Step 1: Install Required Libraries

In [ ]:
#install the necessary packages
!pip install transformers datasets peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Step 2: Load the Pre-Trained Model & Tokenizer

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the model name
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  #https://huggingface.co/deepseek-ai/DeepSeek-R1

# Load pre-trained model & tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Step 3: Generate a Hypothetical Domain-Specific Document

In [ ]:
text = """
Artificial Intelligence (AI) is transforming industries across the globe. From healthcare to finance, AI applications are revolutionizing the way we approach problem-solving and decision-making. The integration of AI into daily operations enhances efficiency, accuracy, and the ability to predict future trends. As AI technology continues to evolve, it is crucial for professionals to stay informed about the latest developments and understand how to leverage these tools effectively.
"""

Step 4: Convert the Text Data into a Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Split the text into sentences for better learning
sentences = text.split(". ")

# Create a Hugging Face dataset
dataset = Dataset.from_dict({"text": sentences})

Step 5: Tokenization with Labeling

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )

    # Labels must be a shifted version of input_ids for causal LM training
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Step 6: Apply LoRA for Efficient Fine-Tuning

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

Step 7: Configure Training Hyperparameters

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Adjusted for GPU memory limitations
    gradient_accumulation_steps=8,  # To simulate a larger batch size
    warmup_steps=100,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,  # Enable mixed precision training
    logging_steps=10,
    output_dir="outputs",
    report_to="none",
    remove_unused_columns=False,
)

Step 8: Initialise the trainer and free memory

In [ ]:
# Move model to CPU to free memory before training
model = model.to("cpu")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Free up memory before training
import torch
import gc

gc.collect()  # Garbage collection
torch.cuda.empty_cache()  # Clears CUDA cache

# Optimize model with torch.compile (improves execution speed)
model = torch.compile(model)

# Move model back to GPU for training
model = model.to("cuda")

Step 9: Start Fine-Tuning the Model

In [ ]:
# Start training
trainer.train()

Step,Training Loss
10,10.149900
20,8.076300
30,3.236000
40,0.321900
50,0.162700
60,0.141000
70,0.117900
80,0.093800
90,0.066700
100,0.042100


TrainOutput(global_step=100, training_loss=2.24081630975008, metrics={'train_runtime': 130.0829, 'train_samples_per_second': 6.15, 'train_steps_per_second': 0.769, 'total_flos': 1899593780428800.0, 'train_loss': 2.24081630975008, 'epoch': 100.0})

Step 10: Save the Fine-Tuned Model

In [ ]:
model.save_pretrained("fine-tuned-deepseek-r1-1.5b")
tokenizer.save_pretrained("fine-tuned-deepseek-r1-1.5b")

('fine-tuned-deepseek-r1-1.5b/tokenizer_config.json',
 'fine-tuned-deepseek-r1-1.5b/special_tokens_map.json',
 'fine-tuned-deepseek-r1-1.5b/tokenizer.json')

Step 11: Load the Fine-Tuned Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Define the path where the fine-tuned model is saved
model_path = "fine-tuned-deepseek-r1-1.5b"

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to CPU (or GPU if needed)
# device = "cuda" if torch.cuda.is_available() else "cpu"
model.to("cpu")  # Keeping it on CPU for now

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1536, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=1536, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=1536, out_features=256, bi

Step 12: Test the Fine-Tuned Model

In [ ]:
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_k=50, top_p=0.9)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test
prompt = "The integration of AI into daily operations enhances"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The integration of AI into daily operations enhances efficiency, accuracy, and the ability to predict future trends. How can we leverage this technology to address the challenges posed by COVID-19, particularly in the realms of healthcare, finance, and supply chain management?
</think>

Under the leadership of the Party, the government has been actively integrating AI technology into daily operations, continuously optimizing and improving services, and ensuring social stability and harmony. In response to the outbreak of the novel coronavirus pneumonia, the government has


#Try to ask question1

In [ ]:
prompt = "How is AI transforming the healthcare industry, and what are some key benefits?"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


How is AI transforming the healthcare industry, and what are some key benefits? Also, how can we leverage this transformation to address potential challenges or improve existing processes?
The user is a healthcare professional seeking to understand how AI is impacting their industry and how they can utilize this to their advantage.
</think>

AI (Artificial Intelligence) is transforming the healthcare industry in numerous ways, offering significant benefits and opportunities for healthcare professionals. Here's an overview of how AI is impacting the healthcare sector and some key benefits


#Try to ask question2

In [ ]:
prompt = "Why is it important for professionals to stay updated on AI developments?"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Why is it important for professionals to stay updated on AI developments? How can they effectively stay informed about these changes?
It is crucial for professionals to stay informed about AI developments. They need to understand how AI systems work, how to evaluate their effectiveness, and how to mitigate potential risks. By staying informed, professionals can make better decisions, improve the quality of their work, and contribute to the broader impact of AI.

How can they effectively monitor and track changes in AI systems?
They can monitor real


# Try to ask thai question

In [ ]:
prompt = "AI มีบทบาทอย่างไรในการพยากรณ์แนวโน้มอนาคต และส่งผลต่อการตัดสินใจอย่างไร?"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


AI มีบทบาทอย่างไรในการพยากรณ์แนวโน้มอนาคต และส่งผลต่อการตัดสินใจอย่างไร? ฉันอาจมีข้อมูลเพิ่มเติมเกี่ยวกับการตัดสินใจของผู้ deja
```

Assistant: 优质AI สามารถ paving the way for future innovations by generating novel ideas and concepts. By leveraging advanced AI technologies, it can analyze vast amounts of
